In [1]:
import sqlite3
conn = sqlite3.connect('results.db')
c = conn.cursor() 

readable = { "NUMBERJACK":"Toulbar2", "GECODE":"Gecode", "OR_TOOLS":"OR-Tools", "CHOCO":"Choco",
            "JACOP":"JaCoP", "G12":"G12", "GECODE_NAT" : "Native Gecode"}

readableProblems = { "on-call-rostering":"On-Call Rostering", "mspsp":"MSPSP", "soft-queens":"Soft N-Queens",
                    "talent-scheduling":"Talent Scheduling", "photo":"Photo Placement"}
from collections import defaultdict

problemToInstance = defaultdict(list)
c.execute("SELECT Problem, Count(Distinct Instance) as Instances FROM JobResult Group By Problem")
for row in c.fetchall():
    problemToInstance[row[0]] = row[1]

    
c.execute("SELECT COUNT(*) FROM ( SELECT Distinct Instance FROM JobResult )")
res = c.fetchone()
numberProblems = res[0]

print "We tried", numberProblems, "instances."

We tried 28 instances.


In [11]:
# now we do the solver comparison 
problemToInstance = defaultdict(list)
c.execute("SELECT Problem, Count(Distinct Instance) as Instances FROM JobResult Group By Problem")
for row in c.fetchall():
    problemToInstance[row[0]] = row[1]

    
c.execute("SELECT COUNT(*) FROM ( SELECT Distinct Instance FROM JobResult )")
res = c.fetchone()
numberProblems = res[0]

print "We tried", numberProblems, "instances."

scriptFile = open("query-native-solver-comparison-pure-views.sql", 'r')
script = scriptFile.read() 
scriptFile.close()
c.executescript(script)
conn.commit()

scriptFile = open("query-native-solver-comparison-pure.sql",'r')
script = scriptFile.read() 
scriptFile.close()

c.execute(script)

currProblem = ""
print "\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill} }l" + \
 "".join(["d{1.5}"  for i in range(0,1)]) + "cd{1.5}" + "".join(["d{1.1}"  for i in range(0,2)]) + "}"
print "\\toprule"
print '''\\multicolumn{1}{c}{Solver} & \multicolumn{1}{c}{Time (secs)} 
          & \multicolumn{1}{c}{\\# Wins}
          & \multicolumn{1}{c}{Objective} 
          & \multicolumn{1}{c}{\% Solved} & \multicolumn{1}{c}{\% Optimal} \\\\'''
for row in c.fetchall():
    (problem, solverId, solverName, elapsed, elapsedSpan, relElapsed, \
     objective, relObjective, wins, solved, optimally) = row
    if currProblem != problem:
        #print "Starting .... ", problem
        currProblem = problem
        print "\\midrule"
        print "\\multicolumn{2}{l}{" + readableProblems[problem] + " ("+ str(problemToInstance[problem])  + " instances)  }   \\\\"
        print "\\midrule"
    print "  ", readable[solverName], "&", '{0:.2f}'.format(elapsed),\
    "\\quad ("+'{0:.2f}'.format(relElapsed)+")" "&", '{0:.0f}'.format(wins),    \
     "&", '{0:.2f}'.format(objective), "\\quad ("+'{0:.2f}'.format(relObjective)+")", "&", \
    '{0:.2f}'.format(solved), "&",'{0:.2f}'.format(optimally), "\\\\"
print "\\bottomrule"
print "\\end{tabular*}"

We tried 28 instances.
\begin{tabular*}{\textwidth}{@{\extracolsep{\fill} }ld{1.5}cd{1.5}d{1.1}d{1.1}}
\toprule
\multicolumn{1}{c}{Solver} & \multicolumn{1}{c}{Time (secs)} 
          & \multicolumn{1}{c}{\# Wins}
          & \multicolumn{1}{c}{Objective} 
          & \multicolumn{1}{c}{\% Solved} & \multicolumn{1}{c}{\% Optimal} \\
\midrule
\multicolumn{2}{l}{MSPSP (8 instances)  }   \\
\midrule
   Gecode & 0.32 \quad (1.00)& 8 & 2.50 \quad (0.00) & 100.00 & 100.00 \\
   G12 & 0.32 \quad (1.01)& 0 & 2.50 \quad (0.00) & 100.00 & 100.00 \\
   OR-Tools & 0.33 \quad (1.05)& 0 & 2.50 \quad (0.00) & 100.00 & 100.00 \\
   JaCoP & 0.52 \quad (1.73)& 0 & 2.50 \quad (0.00) & 100.00 & 100.00 \\
   Choco & 0.70 \quad (2.46)& 0 & 2.50 \quad (0.00) & 100.00 & 100.00 \\
   Toulbar2 & 312.56 \quad (1052.07)& 0 & 29.13 \quad (26.63) & 0.00 & 0.00 \\
\midrule
\multicolumn{2}{l}{On-Call Rostering (7 instances)  }   \\
\midrule
   Toulbar2 & 40.73 \quad (1.44)& 3 & 1.57 \quad (0.00) & 100.00 & 100.00 \\


In [21]:


scriptFile = open("query-native-vs-strictbab-overhead-views.sql",'r')
script = scriptFile.read() 
scriptFile.close()

c.executescript(script)
conn.commit()

# now we do the solver comparison 
problemToInstance = defaultdict(list)
c.execute("SELECT Problem, Count(Distinct Instance) as Instances FROM PvsNativeSummary Group By Problem")
for row in c.fetchall():
    problemToInstance[row[0]] = row[1]

    
c.execute("SELECT COUNT(*) FROM ( SELECT Distinct Instance FROM PvsNativeSummary )")
res = c.fetchone()
numberProblems = res[0]

print "We tried", numberProblems, "instances."

scriptFile = open("query-native-vs-strictbab-overhead.sql",'r')
script = scriptFile.read() 
scriptFile.close()

currProblem = ""
print "\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill} }l" + \
 "".join(["d{1.1}"  for i in range(0,5)]) + "}"
print "\\toprule"
print '''\\multicolumn{1}{c}{Solver} & \multicolumn{1}{c}{Time Smyth} 
          & \multicolumn{1}{c}{Time Weighted} 
          & \multicolumn{1}{c}{Time Toulbar2}
          & \multicolumn{1}{c}{Obj. Weights} & \multicolumn{1}{c}{Obj. Smyth}   \\\\'''

c.execute(script)

def boldify(floatStr):
    split_num = floatStr.split('.')
    return "\\textbf{" + split_num[0]+"}.\\textbf{"+split_num[1] + "}"


for row in c.fetchall():
    (problem, solverName, elapsedSmyth, elapsedWeights, absoluteOverhead, relOverhead, weightsObj, smythObj, elapsedTb) \
    = row
    if currProblem != problem:
        #print "Starting .... ", problem
        currProblem = problem
        print "\\midrule"
        print "\\multicolumn{2}{l}{" + readableProblems[problem] + " ("+ str(problemToInstance[problem])  + " instances)  }   \\\\"
        print "\\midrule"
    
    if elapsedSmyth < elapsedWeights:
        elapsedSmythText = boldify('{0:.2f}'.format(elapsedSmyth)) 
        elapsedWeightsText = '{0:.2f}'.format(elapsedWeights)
    else:
        elapsedWeightsText = boldify('{0:.2f}'.format(elapsedWeights)) 
        elapsedSmythText = '{0:.2f}'.format(elapsedSmyth)
        
    print "  ", readable[solverName], \
    "&",  elapsedSmythText,\
    "&", elapsedWeightsText, "&", \
    "\\emph{-}" if (currProblem == "mspsp" or currProblem == "talent-scheduling") \
    else "\\emph{" + '{0:.2f}'.format(elapsedTb) + "}", \
    "&",    '{0:.2f}'.format(weightsObj), "&", '{0:.2f}'.format(smythObj), "\\\\"
currProblem = ""

print "\\bottomrule"
print "\\end{tabular*}"

We tried 23 instances.
\begin{tabular*}{\textwidth}{@{\extracolsep{\fill} }ld{1.1}d{1.1}d{1.1}d{1.1}d{1.1}}
\toprule
\multicolumn{1}{c}{Solver} & \multicolumn{1}{c}{Time Smyth} 
          & \multicolumn{1}{c}{Time Weighted} 
          & \multicolumn{1}{c}{Time Toulbar2}
          & \multicolumn{1}{c}{Obj. Weights} & \multicolumn{1}{c}{Obj. Smyth}   \\
\midrule
\multicolumn{2}{l}{MSPSP (6 instances)  }   \\
\midrule
   Gecode & 12.74 & \textbf{0}.\textbf{34} & \emph{-} & 2.67 & 5.50 \\
   Native Gecode & 7.82 & \textbf{0}.\textbf{26} & \emph{-} & 2.80 & 5.80 \\
   JaCoP & 4.18 & \textbf{0}.\textbf{45} & \emph{-} & 2.00 & 6.00 \\
\midrule
\multicolumn{2}{l}{On-Call Rostering (5 instances)  }   \\
\midrule
   Gecode & 220.46 & \textbf{133}.\textbf{32} & \emph{14.52} & 3.20 & 7.20 \\
   Native Gecode & 192.50 & \textbf{133}.\textbf{32} & \emph{14.52} & 3.20 & 25.20 \\
   JaCoP & 194.06 & \textbf{135}.\textbf{28} & \emph{14.52} & 3.20 & 26.80 \\
\midrule
\multicolumn{2}{l}{Photo Placement (

In [12]:

conn.close()

In [17]:
print ["d{}"  for i in range(0,5)]

['d{}', 'd{}', 'd{}', 'd{}', 'd{}']


In [20]:

print boldify("3.15")

\textbf{3}.\textbf{15}
